In [1]:
from pathlib import Path

import numpy as np

from utils.config_manager import Config
from model.factory import tts_ljspeech, tts_custom
from data.audio import Audio



2021-11-02 22:53:18.407307: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
config_path = 'config/peeter_jutustav_16/session_paths.yaml'
config_loader = Config(config_path=config_path)
model = config_loader.load_model(None)  # if None defaults to latest
conf = config_loader.config
audio = Audio(conf)

2021-11-02 22:53:20.026400: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-11-02 22:53:20.058870: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-02 22:53:20.059470: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1070 computeCapability: 6.1
coreClock: 1.835GHz coreCount: 15 deviceMemorySize: 7.92GiB deviceMemoryBandwidth: 238.66GiB/s
2021-11-02 22:53:20.059490: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-11-02 22:53:20.062473: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-11-02 22:53:20.062580: I tensorflow/stream_executor/pla

restored weights from logs/peeter_jutustav_16/ljspeech/tts_config_est.aligner_config/weights/ckpt-312 at step 260000


In [3]:
text = 'Karu oli hoolimatu, lampjalgne ja räpane.'

In [4]:
# Phonemize
phons = model.text_pipeline.phonemizer(text)
print(phons)

karʊ oʎɪ hoːʎimatu,lampjalɡne ja ræpane.


In [5]:
# Tokenize
tokens = model.text_pipeline.tokenizer(phons)
print(tokens)

[21, 12, 28, 98, 1, 25, 102, 72, 1, 18, 25, 119, 102, 19, 23, 12, 30, 31, 6, 22, 12, 23, 26, 20, 12, 22, 64, 24, 16, 1, 20, 12, 1, 28, 37, 26, 12, 24, 16, 8]


In [6]:
# Run model
out = model.predict(tokens, encode=False, phoneme_max_duration=None)
mel = out['mel'].numpy().T
print(mel.shape)

2021-11-02 22:53:21.678167: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-11-02 22:53:21.865934: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-11-02 22:53:21.918556: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-11-02 22:53:22.134282: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201


(80, 178)


In [7]:
# Create Mel basis
from librosa import filters
mel_basis = filters.mel(
    sr = audio.config['sampling_rate'],
    n_fft=audio.config['n_fft'],
    n_mels=audio.config['mel_channels'],
    fmin=audio.config['f_min'],
    fmax=audio.config['f_max']
)
print(mel_basis.shape)

(80, 513)


In [8]:
amp_mel = audio._denormalize(mel)
csv_path = '/home/egert/Prog/TTS-CPP/amp.csv'
amp_mel = inverse = np.loadtxt(csv_path, delimiter=',')
print(amp_mel[79][0])

OSError: /home/egert/Prog/TTS-CPP/amp.csv not found.

In [ ]:

# Compute NNLS

csvPath = '/home/egert/Prog/TransformerTTS/outputs/test/inverse.csv'

from librosa.util import nnls
amp_mel = audio._denormalize(mel)
csv_path = '/home/egert/Prog/TTS-CPP/amp.csv'
#amp_mel = inverse = np.loadtxt(csv_path, delimiter=',')
inverse = nnls(mel_basis, amp_mel)
np.savetxt(csvPath, inverse, delimiter=',')
print(inverse.shape)
np.power(inverse, 1./1, out=inverse)



[WARNING] Unresolved object in checkpoint: (root).step


[WARNING] Unresolved object in checkpoint: (root).optimizer.beta_1


[WARNING] Unresolved object in checkpoint: (root).optimizer.beta_2


[WARNING] Unresolved object in checkpoint: (root).optimizer.decay


[WARNING] Unresolved object in checkpoint: (root).optimizer.learning_rate


[WARNING] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/guide/checkpoint#loading_mechanics for details.


(513, 178)


array([[0.0000000e+00, 1.1102230e-16, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.0077699e-01, 1.6119997e-01, 3.3265513e-01, ..., 2.4983598e-01,
        2.1888641e-01, 1.3103242e-01],
       [2.0155399e-01, 3.2239994e-01, 6.6531026e-01, ..., 4.9967197e-01,
        4.3777281e-01, 2.6206484e-01],
       ...,
       [7.2351228e-05, 7.3477560e-05, 4.8982922e-04, ..., 1.4985789e-04,
        1.0030346e-04, 7.9534533e-05],
       [3.6175614e-05, 3.6738780e-05, 2.4491461e-04, ..., 7.4928947e-05,
        5.0151732e-05, 3.9767267e-05],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [9]:
csv_path = '/home/egert/Prog/TTS-CPP/temp/inverse.csv'
inverse = np.loadtxt(csv_path, delimiter=',')
print(inverse.shape)

(513, 178)


In [10]:
# Griffin-lim
from librosa.core import griffinlim
wav = griffinlim(
    inverse,
    n_iter = 32,
    hop_length=audio.config['hop_length'],
    win_length=audio.config['win_length'],
)

(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)
(45312,)


In [ ]:
outdir = Path('outputs/test')
outdir.mkdir(exist_ok=True, parents=True)
audio.save_wav(wav, (outdir / f"test").with_suffix('.wav'))